In [ ]:
#You WILL need the following in bash:
#    pip install numpy pandas matplotlib seaborn scikit-learn

# MAKE SURE EVERYTHING IS UP TO DATE IF PREDOWNLOADED!!!


In [17]:
# General things for functions
import numpy as np # Numerical commands
import pandas as pd # Data manipulation

#Visualization tools for plotting
import matplotlib.pyplot as plt 
import seaborn as sns


#Machine learning parts itself
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Importing of the files. This will take a WHILE
testAway = pd.read_csv('testDataV3_away.txt')
testHome = pd.read_csv('testDataV3_home.txt')
testY = pd.read_csv('testDataV3_y.txt')

trainAway = pd.read_csv('trainDataV3_away.txt')
trainHome = pd.read_csv('trainDataV3_home.txt')
trainY = pd.read_csv('trainDataV3_y.txt')

In [ ]:
def calculate_game_stats(df):
    # This will calculate the total number of games played
    df['TOTAL_GAMES_PLAYED'] = df['TEAM_GAME_WON_season_sum'] + df['TEAM_GAME_DRAW_season_sum'] + df['TEAM_GAME_LOST_season_sum']
    
    # Calculate Win perctengae: Wins / (total of all games)
    # This is of course to determine "good" compared to "bad" teams and see what factors are different
    df['WIN_PERCENTAGE'] = (df['TEAM_GAME_WON_season_sum'] / df['TOTAL_GAMES_PLAYED']) *100
    
    return df


trainHome = calculate_game_stats(trainHome)
trainAway = calculate_game_stats(trainAway)
testHome = calculate_game_stats(testHome)
testAway = calculate_game_stats(testAway)


print("Train Home Data with Stats:")
print(trainHome[['TEAM_GAME_WON_season_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_GAME_LOST_season_sum', 'TOTAL_GAMES_PLAYED', 'WIN_PERCENTAGE']].head())

print("\nTrain Away Data with Stats:")
print(trainAway[['TEAM_GAME_WON_season_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_GAME_LOST_season_sum', 'TOTAL_GAMES_PLAYED', 'WIN_PERCENTAGE']].head())


# Below was a simple test to ensure the test data win loss columns were created
# This is not to be used until model evaluation with the training data

# print("\nTest Home Data with Stats:")
# print(testHome[['TEAM_GAME_WON_season_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_GAME_LOST_season_sum', 'TOTAL_GAMES_PLAYED', 'WIN_LOSS_RATIO']].head())

# print("\nTest Away Data with Stats:")
# print(testAway[['TEAM_GAME_WON_season_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_GAME_LOST_season_sum', 'TOTAL_GAMES_PLAYED', 'WIN_LOSS_RATIO']].head())

Train Home Data with Stats:
   TEAM_GAME_WON_season_sum  TEAM_GAME_DRAW_season_sum  \
0                       1.0                        6.0   
1                       8.0                        4.0   
2                       2.0                        4.0   
3                       7.0                        0.0   
4                       1.0                        6.0   

   TEAM_GAME_LOST_season_sum  TOTAL_GAMES_PLAYED  WIN_PERCENTAGE  
0                       10.0                17.0        5.882353  
1                        1.0                13.0       61.538462  
2                        8.0                14.0       14.285714  
3                        5.0                12.0       58.333333  
4                       10.0                17.0        5.882353  

Train Away Data with Stats:
   TEAM_GAME_WON_season_sum  TEAM_GAME_DRAW_season_sum  \
0                       5.0                        8.0   
1                       1.0                        4.0   
2                 